In [2]:
#Libraries
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt 

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

#VGG16
from keras.applications.vgg16 import VGG16

#resnet Model
from keras.applications.resnet50 import ResNet50

#GoogLeNet/Inception
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [3]:
file_path = 'C:/Users/hhaggan/Desktop/BehaviorCloning/data/data/'

def GetData():
    lines = []
    images = []
    center = []
    left = []
    right = []
    measurements = []

    with open(file_path + 'driving_log.csv') as csvFile:
        reader = csv.reader(csvFile)
        next(reader, None)
        for line in reader:
            steering_center = float(line[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.2 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            measurements.append(float(line[3]))
            center.append(file_path + '/' + line[0].strip())
            left.append(file_path + '/' + line[1].strip())
            right.append(file_path + '/' + line[2].strip())
    
    images.extend(center)
    images.extend(left)
    images.extend(right)
    
    return images, measurements

In [ ]:
#data augmentation
def Augment_Data(images_path, measurements):
    augmented_images, augmented_measurements = [], []
    for image_path, measurement in zip(images_path, measurements):
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        augmented_images.append(cv2.flip(image, 1))
        augmented_measurements.append(measurement*-1.0)
    return augmented_images, augmented_measurements

In [5]:
#building NVIdia model
def Nvidia_Model():
    
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((75, 25), (0,0))))
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    #model.add(MaxPooling2D())
    model.add(Convolution2D(36,5,5, subsample=(2,2),activation='relu'))
    #model.add(MaxPooling2D())
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3,activation='relu'))
    model.add(Convolution2D(64,3,3,activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    return model

In [4]:
def Train_Nvidia_Model(model, X_train, y_train):
    model.compile(loss='mse', optimizer='adam')
    model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

    model.save('nvidiamodel.h5')

In [8]:
    
def ResNet_Model():
    from keras.applications.resnet50 import preprocess_input, decode_predictions
    
    model = ResNet50(weights='imagenet', include_top=False)
    preprocess_input(X_train)

    return model

In [6]:
def Train_ResNet_Model(model, X_train, y_train):

    model.save('ResNetmodel.h5')

SyntaxError: unexpected EOF while parsing (<ipython-input-6-ae98548590b3>, line 2)

In [ ]:
def Inception_Model():
    model = InceptionV3(weights='imagenet', include_top=False)
    
    return model

In [ ]:
def Train_Inception_Model(model, X_train, y_train):
    model.save('Inceptionmodel.h5')

In [ ]:
def VGG_Model():
    model = VGG16(weights='imagenet', include_top=False)
    
    return model

In [ ]:
def Train_VGG_Model(model, X_train, y_train):
    model.save('VGGmodel.h5')

In [7]:
def __main__():
    print('Getting the data for the images and the steering measurements...')
    image_path, measurements = GetData()
    print('Augmenting the existing data...')
    images, measurements = Augment_Data(image_path, measurements) 
    
    X_train = np.array(images)
    y_train = np.array(measurements)
    
    '''print('Training VGG Model') 
    VGGModel = VGGModel()
    Train_VGG_Model(VGGModel)
    
    print('Training an Inception Model')
    InceptionModel = Inception_Model()
    Train_Inception_Model(InceptionModel)
    
    print('Training ResNet Model')
    ResNetModel = ResNet_Model()
    Train_ResNet_Model(ResNetModel)'''
    
    print('Training a Nvidia Model')
    NvidiaModel = Nvidia_Model()
    Train_Nvidia_Model(NvidiaModel)

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation

ValueError: Error when checking input: expected lambda_1_input to have 4 dimensions, but got array with shape (0, 1)

In [6]:
X_train = np.array(images)
y_train = np.array(measurements)

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70, 25), (0,0))))
model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(36,5,5, subsample=(2,2),activation='relu'))
#model.add(MaxPooling2D())
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation

ValueError: Error when checking input: expected lambda_2_input to have 4 dimensions, but got array with shape (0, 1)